In [51]:
import pandas as pd
import cv2
import os
import json
# Only include the following code if you are running the complete cloned project
# while os.getcwd().split('/')[-1] != 'mmpose-synthetic-tune':
#     os.chdir('..')

In [45]:
DLC_LABELS_PATH = 'DLC_Labels.csv'
labels_csv = pd.read_csv(DLC_LABELS_PATH, header=None)
DLC_keypoints = labels_csv.iloc[1][3:].values
labels_csv.fillna(-1, inplace=True)

csv_header = ['image_name', *DLC_keypoints]

labels = labels_csv.iloc[3:, 2:]
labels = pd.DataFrame(labels.values, columns=csv_header)
labels.head()

,image_name,Back1,Back1,Back2,Back2,Back3,Back3,Back4,Back4,Head1,...,LHF5,LHF5,RHF2,RHF2,RHF3,RHF3,RHF5,RHF5,Tail1,Tail1
0,img000.png,203.0,317.0,335.0,319.0,479.0,316.0,593.0,321.0,663.0,...,165.0,671.0,251.0,497.0,271.0,576.0,-1,-1,164.0,532.0
1,img001.png,206.0,317.0,337.0,319.0,482.0,316.0,596.0,320.0,666.0,...,165.0,671.0,254.0,497.0,272.0,576.0,-1,-1,165.0,532.0
2,img002.png,213.0,317.0,345.0,320.0,488.0,316.0,604.0,318.0,675.0,...,167.0,671.0,262.0,497.0,274.0,577.0,-1,-1,168.0,533.0
3,img003.png,224.0,317.0,355.0,320.0,496.0,316.0,614.0,315.0,687.0,...,168.0,671.0,272.0,497.0,277.0,577.0,-1,-1,173.0,533.0
4,img004.png,235.0,317.0,366.0,320.0,506.0,316.0,626.0,313.0,699.0,...,170.0,670.0,283.0,498.0,280.0,577.0,-1,-1,180.0,533.0


In [31]:
def pairwise(iterable):
    return zip(*[iter(iterable)]*2)

In [49]:
COCO_keypoints = [ DLC_keypoints[i] for i in range(0, len(DLC_keypoints), 2) ]
dataset = {
    "categories": [
        {
            "id": 1,
            "name": "", # Name your category here
            "supercategory": "",
            "keypoints": COCO_keypoints,
            "skeleton": [] # Fit in the skeleton by your definition: [ [1, 2], [2, 3], ... ]
        }
    ],
    "images": [],
    "annotations": []
}

In [53]:
images = []
annotations = []
last_image_id = 0
dataset_image_size = (1280, 720) # Fill this part according to your data
(w, h) = dataset_image_size

for r in labels.iterrows():
    last_image_id += 1

    entry = r[1]
    
    img_name: str = entry['image_name']

    keypoints = entry.iloc[1:].astype(float).astype(int)

    # Adding image info
    image_info = {
        'license': 1,
        "id": last_image_id,
        "file_name": img_name,
        "width": w,
        "height": h,
        "background": 1
    }

    # Adding annotation info
    annotation_info = {
        "id": last_image_id,
        "image_id": last_image_id,
        "category_id": 1,
        "bbox": [],
        'segmentation': [],
        "area": 0,
        "iscrowd": 0,
        "num_keypoints": len(COCO_keypoints),
        "keypoints": []
    }

    # Adding bbox info
    annotation_info["bbox"] = [0, 0, w, h]
    annotation_info["area"] = w * h

    # Adding keypoints
    annotation_kps = []
    for idx, ((_, x), (_, y)) in enumerate(pairwise(keypoints.items())):
        if idx == 20:
            break

        if x != -1 and y != -1:
            annotation_kps.extend([x, y, 2])
        else:
            annotation_kps.extend([0, 0, 0])

    annotation_info['keypoints'] = annotation_kps

    images.append(image_info)
    annotations.append(annotation_info)

dataset['images'].extend(images)
dataset['annotations'].extend(annotations)

with open(DLC_LABELS_PATH.replace('.csv', '_COCO.json'), 'w') as f:
    f.write(json.dumps(dataset))